In [11]:
import numpy as np
import sobol_seq
import matplotlib.pyplot as plt
from functools import partial
from typing import List, Optional, Callable, Tuple


#### Problem TDS
Source: https://towardsdatascience.com/differential-evolution-an-alternative-to-nonlinear-convex-optimization-690a123f3413

In [12]:
# NP = 50 # number of population
# FEs_max = 2000 # maximum number of fitness evaluation
# F = 0.3 # mutation factor
# CR = 0.7 # crossover rate

# dim = 2
# boundaries = np.array([(-5,5) for _ in range (dim)])
# np.random.seed(0)

# def objective_function(x):
#     f = (x[0] - 0.5) ** 2 + 0.7 * x[0] * x[1] + 1.2 * (x[1] + 0.7) ** 2
#     return f



#### Problem Random

In [13]:
# NP = 50 # number of population
# FEs_max = 2000 # maximum number of fitness evaluation
# F = 0.3 # mutation factor
# CR = 0.7 # crossover rate

# dim = 2
# boundaries = np.array([(-20,20) for _ in range (dim)])
# np.random.seed(0)

# def objective_function(x):
#     z = (x[0]+10)**2 +(x[1]-5)**2
#     return z

#### Problem Tamura

In [14]:
NP = 20 # number of population
gen_max = 100 # maximum number of fitness evaluation
F = 0.3 # mutation factor
CR = 0.7 # crossover rate

dim = 2
boundaries = np.array([(-20,20) for _ in range (dim)])
np.random.seed(0)

def objective_function(x): # x:tuple n-dimension
    f = 0
    """Schwefel"""
    for i in range (len(x)):
        sum_sq = 0
        for j in range (i+1):
            sum_sq += x[j]
        f += sum_sq**2 
    """2^n Minima"""
    # for i in range (len(x)):
    #     f += (x[i]**4-16*x[i]**2+5*x[i])
    """Rastrigin"""
    # for i in range (len(x)):
    #     f += (x[i]**2-10*np.cos(2*np.pi*x[i])+10)
    return f

In [15]:
"""GENERATE POINTS USING SOBOL SEQUENCE"""
def generate_points(dim,npoint,low=-10,high=10):
    if type(low) != type(high):
        raise TypeError('The type of "low" and "high" should be the same.')
    if type(low) == int:
        boundaries = [(low,high) for _ in range (dim)]
    elif type(low) == list or type(low) == np.ndarray:
        if len(low) != len(high):
            raise TypeError('The length of "low" and "high" should be the same.')
        else:
            boundaries = [(low[i],high[i]) for i in range (len(low))]

    # Generate Sobol sequence points
    sobol_points = sobol_seq.i4_sobol_generate(dim, npoint)

    # Scale the Sobol points to fit within the specified boundaries
    scaled_points = []
    for i in range(dim):
        a, b = boundaries[i]
        scaled_dim = a + sobol_points[:, i] * (b - a)
        scaled_points.append(scaled_dim)

    # Transpose the scaled points to get points per dimension
    scaled_points = np.array(list(map(list, zip(*scaled_points))))
    return scaled_points


In [16]:
popA = generate_points(dim=dim,
                        npoint=NP,
                        low=boundaries[:,0],
                        high=boundaries[:,1])
genA = {0:popA}
print(genA)

{0: array([[  0.  ,   0.  ],
       [ 10.  , -10.  ],
       [-10.  ,  10.  ],
       [ -5.  ,  -5.  ],
       [ 15.  ,  15.  ],
       [  5.  , -15.  ],
       [-15.  ,   5.  ],
       [-12.5 ,  -7.5 ],
       [  7.5 ,  12.5 ],
       [ 17.5 , -17.5 ],
       [ -2.5 ,   2.5 ],
       [ -7.5 , -12.5 ],
       [ 12.5 ,   7.5 ],
       [  2.5 ,  -2.5 ],
       [-17.5 ,  17.5 ],
       [-16.25,  -1.25],
       [  3.75,  18.75],
       [ 13.75, -11.25],
       [ -6.25,   8.75],
       [ -1.25, -16.25]])}


In [17]:
def crossover(individual:np.ndarray, honor_vector:np.ndarray):
    trial_vector = np.zeros(shape=individual.shape)
    for j in range (individual.shape[0]):
        rand_j = np.random.random() # a uniformly distributed random number from [0, 1]
        j_rand = np.random.randint(dim+1) #a random integer uniformly generated from {1, . . . , n}
        if (rand_j<CR) | (j == j_rand):
            trial_vector[j] = honor_vector[j]
        else: #(rand_j>=CR) and (j != j_rand)
            trial_vector[j] = individual[j]
    return trial_vector

# X = np.array([5,6])
# V = np.array([121,343])
# crossover(X,V)

In [18]:
def mutation(xr1,xr2,xr3,scaling_factor):
    donor_vector = xr1 + scaling_factor*(xr2-xr3)
    return donor_vector

In [19]:
np.random.seed(0)
FEs = 1#0
DV = [] #donor vector array
G = 0
while G<gen_max:
    popB = genA[G].copy()
    pop_ids = np.arange(popB.shape[0])
    for i in range (NP):
        x_i = popB[i]
        """GENERATE: three distinct individuals xr1, xr1, xr1 from the current population randomly"""
        pop_ids_no_i = np.delete(pop_ids,i)
        xr1,xr2,xr3 = popB[np.random.choice(pop_ids_no_i,3,replace=False)] 

        """MUTATION: Form the donor/mutation vector"""
        dv_i = mutation(xr1,xr2,xr3,F)
        DV.append(dv_i)

        """CROSSOVER: The trial vector ui is developed either from the elements of the target vector xi or the elements of the donor vector vi"""
        tv_i = crossover(x_i,dv_i)

        """EVALUATE: If f (ui) ≤ f (xi) then replace the individual xi in the population with the trial vector ui"""
        if objective_function(tv_i)<=objective_function(x_i):
            popB[i] = tv_i

    G += 1
    genA[G] = popB
print(genA)

{0: array([[  0.  ,   0.  ],
       [ 10.  , -10.  ],
       [-10.  ,  10.  ],
       [ -5.  ,  -5.  ],
       [ 15.  ,  15.  ],
       [  5.  , -15.  ],
       [-15.  ,   5.  ],
       [-12.5 ,  -7.5 ],
       [  7.5 ,  12.5 ],
       [ 17.5 , -17.5 ],
       [ -2.5 ,   2.5 ],
       [ -7.5 , -12.5 ],
       [ 12.5 ,   7.5 ],
       [  2.5 ,  -2.5 ],
       [-17.5 ,  17.5 ],
       [-16.25,  -1.25],
       [  3.75,  18.75],
       [ 13.75, -11.25],
       [ -6.25,   8.75],
       [ -1.25, -16.25]]), 1: array([[  0.    ,   0.    ],
       [ 10.    , -10.    ],
       [-10.    ,  10.    ],
       [ -5.    ,  -5.    ],
       [ 15.    ,  15.    ],
       [  5.    , -15.    ],
       [-15.    ,   5.    ],
       [ -0.625 ,   1.375 ],
       [  7.5   ,  12.5   ],
       [ 17.5   , -17.5   ],
       [ -2.5   ,   2.5   ],
       [-10.    ,  10.    ],
       [ -5.625 ,   0.375 ],
       [  2.5   ,  -2.5   ],
       [-17.5   ,  17.5   ],
       [-16.25  ,  -1.25  ],
       [  3.75  ,  18.75  ]

# Animation

In [20]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display

# Create a Matplotlib figure and axis
fig, ax = plt.subplots()
ax.set_xlim(boundaries[0])
ax.set_ylim(boundaries[1])
ax.grid(True)
# ax.plot()
ax.autoscale()

# Initialize an empty scatter plot (it will be updated in the animation)
sc = ax.scatter([], [])

points = genA[0]
sc.set_offsets(points)

# Define an initialization function
def init():
    return sc,

# Define the update function for the animation
def update(frame, points):
    points = genA[frame]
    sc.set_offsets(points)
    ax.autoscale_view()
    return sc,

# Create the animation and pass points as an argument to the update function
ani = FuncAnimation(fig, update, frames=len(genA), init_func=init, fargs=(points,), blit=True)

ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.set_title('Animation of Arbitrary Number of Points')

# Close the previous figure
plt.close()

# Display the animation as HTML
display(HTML(ani.to_jshtml()))


## Automatic

In [24]:
import deal 

# popA = deal.generate_points(dim=dim,
#                         npoint=20,
#                         low=boundaries[:,0],
#                         high=boundaries[:,1])
# genA = {0:popA}
# deal.reproduction(population=popA,
#                   objective_function=objective_function,
#                   dimension=dim,
#                   mutation_factor=F,
#                   crossover_rate=CR)
genA = deal.differensial_evolution(number_of_population=20,
                                    objective_function=objective_function,
                                    boundaries=boundaries,
                                    dimension=dim,
                                    gen_max=gen_max,
                                    mutation_factor=F,
                                    crossover_rate=CR,
                                    seed=0)


array([0., 0.])